In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Including Dependencies

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import cv2
import os
import glob
import pickle

from random import sample, randint

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Flatten,Conv2D,MaxPool2D,Dropout
from keras.applications.vgg19 import VGG19

from lxml import etree

Reading Data Provided

In [ ]:
for dirName, _, fileNames in os.walk('/content/drive/MyDrive/car plate recoginition/car_cnn'):
    for fileName in fileNames:
        print(os.path.join(dirName, fileName))

Feature Extraction

In [ ]:
# x data contains car images
# Set Image Size
IMAGE_SIZE = 200

img_dir = "/content/drive/MyDrive/car plate recoginition/car_cnn/images" # Enter Directory of all images 
data_path = os.path.join(img_dir,'*g')

files = glob.glob(data_path)
files.sort()

X=[]
for f1 in files:
    img = cv2.imread(f1)
    img = cv2.resize(img, (IMAGE_SIZE,IMAGE_SIZE))
    X.append(np.array(img))

In [ ]:
print(X[0].shape)

In [ ]:
def resizeannotation(f):
    tree = etree.parse(f)
    for dim in tree.xpath("size"):
        width = int(dim.xpath("width")[0].text)
        height = int(dim.xpath("height")[0].text)
    for dim in tree.xpath("object/bndbox"):
        xmin = int(dim.xpath("xmin")[0].text)/(width/IMAGE_SIZE)
        ymin = int(dim.xpath("ymin")[0].text)/(height/IMAGE_SIZE)
        xmax = int(dim.xpath("xmax")[0].text)/(width/IMAGE_SIZE)
        ymax = int(dim.xpath("ymax")[0].text)/(height/IMAGE_SIZE)
    return [int(xmax), int(ymax), int(xmin), int(ymin)]

In [ ]:
#y data contains car annotations
path = '/content/drive/MyDrive/car plate recoginition/car_cnn/annotations/'
text_files = ['/content/drive/MyDrive/car plate recoginition/car_cnn/annotations/'+f for f in sorted(os.listdir(path))]
y=[]
for i in text_files:
    y.append(resizeannotation(i))

In [ ]:
print(f"X Shape : {np.array(X).shape}")
print(f"y Shape : {np.array(y).shape}")

Data Sampling

In [ ]:
sample_index = []

for i in range(6):
    sample_index.append(randint(0, len(X)))
    
plt.figure(figsize=(15, 15))
for index, i in enumerate(sample_index):
    plt.subplot(3, 3, index + 1)
    image = cv2.rectangle(cv2.cvtColor(X[i], cv2.COLOR_BGR2RGB),(y[i][0],y[i][1]),(y[i][2],y[i][3]),(0, 0, 255))
    plt.imshow(image)
    plt.axis("off")

plt.show()

In [ ]:
for i in range(6):
    print(y[i])
    

Data Preprocessing

In [ ]:
X = np.array(X)
y = np.array(y)

X = X / 255
y = y / 255

Train and Validation split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape,X_val.shape

Model Creation

In [ ]:
model = Sequential()
model.add(VGG19(weights="imagenet", include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(4, activation="sigmoid"))

model.layers[-6].trainable = False

model.summary()

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=7, batch_size=32, verbose=1)

Model Evaluation on Metrices

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training and validation loss curves')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Training and validation accuracy curves')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
scores = model.evaluate(X_val, y_val, verbose=0)
print(f"Score : {scores[1]* 100}%")

Model Evaluation on Test Image (opencv_car.jpg)

In [ ]:
#Reading and Resizing Image
test=cv2.imread("/content/drive/MyDrive/car plate recoginition/opencv_car.jpg")
test = cv2.resize(test, (IMAGE_SIZE,IMAGE_SIZE))
plt.imshow(test)
#x_test=

In [ ]:
# Image Preprocessing
x_test=np.array(test)
x_test=x_test/255
print(x_test.shape)

xx=x_test
xx = np.expand_dims(xx, axis=0)   #adding dimension
print(xx.shape)

In [ ]:
y_test=model.predict(xx)

In [ ]:
y_test
ny = y_test * 255
ny[0][0]

In [ ]:
image = cv2.rectangle(x_test,(int(ny[0][0]),int(ny[0][1])),(int(ny[0][2]),int(ny[0][3])),(0, 255, 0))
plt.imshow(image)
plt.show()

In [ ]:
test_loss, test_accuracy = model.evaluate(xx, y_test,steps=int(100))
print(f"Loss : {test_loss * 100}%")
print(f"Accuracy : {test_accuracy * 100}%")

In [ ]:
filename = 'vgg19.pkl'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
saved_vgg19_model = pickle.load(open(filename, 'rb'))
loss,accuracy = saved_vgg19_model.evaluate(xx,y_test)
print(f"Loss of VGG19 is :,{loss*100}%")
print(f"Accuracy of VGG19 is :,{accuracy*100}%")

# New Section

-----------------------------------------------------------------------------

CNN Model

In [ ]:
X_train.shape,X_val.shape

In [ ]:
cnnmodel = Sequential()

cnnmodel.add(Conv2D(filters=128,kernel_size=3,activation='relu',input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
cnnmodel.add(Conv2D(filters=64,kernel_size=3,activation='relu'))
cnnmodel.add(Dropout(0.15))
cnnmodel.add(Conv2D(filters=32,kernel_size=3,activation='relu'))
cnnmodel.add(Conv2D(filters=16,kernel_size=3,activation='relu'))





cnnmodel.add(MaxPool2D(pool_size=(2,2)))

cnnmodel.add(Flatten())

cnnmodel.add(Dense(128, activation="relu"))
cnnmodel.add(Dense(128, activation="relu"))
cnnmodel.add(Dense(64, activation="relu"))
cnnmodel.add(Dense(4, activation="sigmoid"))



cnnmodel.summary()

In [ ]:
cnnmodel.compile(loss='mean_squared_error',optimizer='adam',metrics=['accuracy'])
history=cnnmodel.fit(X_train,y_train,validation_data=(X_val,y_val),epochs= 10,batch_size=32,verbose=1)

In [ ]:
scores = cnnmodel.evaluate(X_val, y_val, verbose=0)
print(f"Score : {scores[1]* 100}%")

In [ ]:
#Reading and Resizing Image
test=cv2.imread("/content/drive/MyDrive/car plate recoginition/opencv_car.jpg")
test = cv2.resize(test, (IMAGE_SIZE,IMAGE_SIZE))
plt.imshow(test)

In [ ]:
# Image Preprocessing
x_test=np.array(test)
x_test=x_test/255
print(x_test.shape)

xx=x_test
xx = np.expand_dims(xx, axis=0)   #adding dimension
print(xx.shape)

In [ ]:
y_test=cnnmodel.predict(xx)

In [ ]:
y_test
ny = y_test * 255
ny[0][0]

In [ ]:
image = cv2.rectangle(x_test,(int(ny[0][0]),int(ny[0][1])),(int(ny[0][2]),int(ny[0][3])),(0, 255, 0))
plt.imshow(image)
plt.show()

In [ ]:
filename = 'cnn.pkl'
pickle.dump(cnnmodel, open(filename, 'wb'))

In [ ]:
saved_cnn_model = pickle.load(open(filename, 'rb'))
loss,accuracy = saved_vgg19_model.evaluate(xx,y_test)
print(f"Loss of CNN is :,{loss*100}%")
print(f"Accuracy of CNN is :,{accuracy*100}%")

--------------------------------------------------------------------------------------------------------------------------------

**Resnet50**

In [ ]:
from keras.models import Model, load_model
from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.layers import GlobalAveragePooling2D, Dropout
import tensorflow as tf

In [ ]:
# RESNET as the base model
#resnet_model = Sequential()
resnet_model=tf.keras.applications.resnet50.ResNet50(weights= "imagenet", include_top=False, input_shape= (IMAGE_SIZE,IMAGE_SIZE,3))
for layer in resnet_model.layers:
  layer.trainable=False

# Take the output output of the base layers
base_model_output=resnet_model.output
x=Flatten()(base_model_output)
#x=GlobalAveragePooling2D()()
# FC
x=Dense(128,activation='relu')(x)
x=Dense(128,activation='relu')(x)
x=Dense(64,activation='relu')(x)
## Dropout
x=Dropout(0.5)(x)
## Output layer
x=Dense(4,activation='softmax')(x)






In [ ]:
resnet_model=Model(inputs=resnet_model.input,outputs=x)
resnet_model.summary()

In [ ]:
resnet_model.compile(loss='mean_squared_error',optimizer='adam',metrics=['accuracy'])
history=resnet_model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs= 10,batch_size=32,verbose=1)

In [ ]:
#Reading and Resizing Image
test=cv2.imread("/content/drive/MyDrive/car plate recoginition/opencv_car.jpg")
test = cv2.resize(test, (IMAGE_SIZE,IMAGE_SIZE))
plt.imshow(test)

In [ ]:
# Image Preprocessing
x_test=np.array(test)
x_test=x_test/255
print(x_test.shape)

xx=x_test
xx = np.expand_dims(xx, axis=0)   #adding dimension
print(xx.shape)

In [ ]:
y_test=resnet_model.predict(xx)

In [ ]:
y_test
ny = y_test * 255
ny[0][0]

In [ ]:
image = cv2.rectangle(x_test,(int(ny[0][0]),int(ny[0][1])),(int(ny[0][2]),int(ny[0][3])),(0, 255, 0))
plt.imshow(image)
plt.show()

In [ ]:
filename = 'resnet50.pkl'
pickle.dump(resnet_model, open(filename, 'wb'))

In [ ]:
saved_resnet50_model = pickle.load(open(filename, 'rb'))
loss,accuracy = saved_resnet50_model.evaluate(xx,y_test)
print(f"Loss of CNN is :,{loss*100}%")
print(f"Accuracy of CNN is :,{accuracy*100}%")